<a href="https://colab.research.google.com/github/tsitsvero/rotor-gp/blob/main/pack/train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [18]:
!pip install git+https://github.com/lab-cosmo/librascal gpytorch pytorch-lightning wandb git+https://github.com/chem-gp/fande xtb nbstripout
! nbstripout --install --global

  Cloning https://github.com/lab-cosmo/librascal to /tmp/pip-req-build-ufpjb1v8
  Running command git clone --filter=blob:none --quiet https://github.com/lab-cosmo/librascal /tmp/pip-req-build-ufpjb1v8
  Resolved https://github.com/lab-cosmo/librascal to commit 6c55e99720f9a181f1efc5c7cd4976c87809d79b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/chem-gp/fande to /tmp/pip-req-build-heot65o9
  Running command git clone --filter=blob:none --quiet https://github.com/chem-gp/fande /tmp/pip-req-build-heot65o9
  Resolved https://github.com/chem-gp/fande to commit 353b1733424bfdd15a117ac41dd5f909ed5ce40c
  Preparing metadata (setup.py) ... done


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!wget https://raw.githubusercontent.com/tsitsvero/rotor-gp/main/pack/cook_model.py

--2023-12-31 14:32:33--  https://raw.githubusercontent.com/tsitsvero/rotor-gp/main/pack/cook_model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9070 (8.9K) [text/plain]
Saving to: ‘cook_model.py.1’

cook_model.py.1     100%[===================>]   8.86K  --.-KB/s    in 0s      

2023-12-31 14:32:33 (75.3 MB/s) - ‘cook_model.py.1’ saved [9070/9070]



## Cooking up the model

In [21]:
# Prepare the training data:

from ase import io
import os

# DATA_DIR = os.path.expanduser("~/repos/data/")
# # DATA_DIR = "/data1/simulations/datasets/rotors/high_temp_ML_training_data/"
# RESULTS_DIR = os.path.expanduser("~/repos/data/results")

DATA_DIR = os.path.expanduser("/content/drive/MyDrive/data/")
# FANDE_DIR = os.path.expanduser("~/")
RESULTS_DIR = os.path.expanduser("/content/drive/MyDrive/data/results/")
os.makedirs(RESULTS_DIR, exist_ok=True)

PREPARE_DATA = False
if PREPARE_DATA:
        traj_295 = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index=":")
        # traj_355 = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")
        traj_295_2000K = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
        # traj_355_2000K = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
        # traj_295_2000K_forced = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
        # traj_355_2000K_forced = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")

        # trajectory_energy = traj_295[0:5000] + traj_355[0:5000] + traj_295_2000K[0:5000] + traj_355_2000K[0:5000] + traj_295_2000K_forced[0:5000] + traj_355_2000K_forced[0:5000]

        train_energy = traj_295[0:5000:10] +  traj_295_2000K[0:5000:10]
        train_forces = traj_295_2000K[0:5000:5]
        validation_energy = traj_295[1000:1010]
        validation_forces = traj_295[1000:1010]

        io.write(DATA_DIR+"/train_energy.traj", train_energy)
        io.write(DATA_DIR+"/train_forces.traj", train_forces)
        io.write(DATA_DIR+"/validation_energy.traj", validation_energy)
        io.write(DATA_DIR+"/validation_forces.traj", validation_forces)


In [22]:
%%time
# ENERGY_MODEL = 'variational_inducing_points' #'variational_inducing_points', 'exact'
# ENERGY_NUM_INDUCING_POINTS = 10
# ENERGY_LR = 0.1
# ENERGY_NUM_STEPS = 5

# FORCES_MODEL = 'variational_inducing_points' #'variational_inducing_points', 'exact'
# FORCES_NUM_INDUCING_POINTS = 10
# NUM_FORCE_SAMPLES = 10
# FORCES_LR = 0.1
# FORCES_NUM_STEPS = 5

# PREDICTOR_NAME = 'test.pth'
# SUBSAMPLE = 200 # subsample data to reduce time durings tests


ENERGY_MODEL = 'exact' #'variational_inducing_points', 'exact'
ENERGY_NUM_INDUCING_POINTS = 500
ENERGY_LR = 0.01
ENERGY_NUM_STEPS = 2000

FORCES_MODEL = 'exact' #'variational_inducing_points', 'exact'
FORCES_NUM_INDUCING_POINTS = 500
NUM_FORCE_SAMPLES = 1000
FORCES_LR = 0.01
FORCES_NUM_STEPS = 2000

PREDICTOR_NAME = 'exact.pth'
SUBSAMPLE = 2 # subsample data to reduce time durings tests

! python cook_model.py \
--data_dir $DATA_DIR \
--results_dir $RESULTS_DIR \
--energy_model $ENERGY_MODEL \
--energy_num_inducing_points $ENERGY_NUM_INDUCING_POINTS \
--energy_lr $ENERGY_LR \
--energy_num_steps $ENERGY_NUM_STEPS \
--forces_model $FORCES_MODEL \
--forces_num_inducing_points $FORCES_NUM_INDUCING_POINTS \
--num_force_samples $NUM_FORCE_SAMPLES \
--forces_lr $FORCES_LR \
--forces_num_steps $FORCES_NUM_STEPS \
--predictor_name $PREDICTOR_NAME \
--subsample $SUBSAMPLE


DATA_DIR /content/drive/MyDrive/data/
RESULTS_DIR /content/drive/MyDrive/data/results/
ENERGY_MODEL exact
ENERGY_NUM_INDUCING_POINTS 500
ENERGY_LR 0.01
ENERGY_NUM_STEPS 2000
FORCES_MODEL exact
FORCES_NUM_INDUCING_POINTS 500
NUM_FORCE_SAMPLES 1000
FORCES_LR 0.01
FORCES_NUM_STEPS 2000
PREDICTOR_NAME exact.pth
500 500
fande.models module imported...
Icecream logger is not available
Total number of found groups: 14
Checking if all atoms are covered:  True
Preparing dataloaders...
Total length of traj is 500
Total number of batches 500
Calculating invariants on trajectory with librascal...
100% 500/500 [00:21<00:00, 23.33it/s]
invariants for energy fitting calculated
100% 500/500 [13:25<00:00,  1.61s/it]
invariants for forces fitting calculated
torch.Size([500, 1200])
torch.Size([500])
Dataloader for group 0 created
Number of samples in dataloader: 1000
Dataloader for group 1 created
Number of samples in dataloader: 1000
Dataloader for group 2 created
Number of samples in dataloader: 1000
D

In [23]:
%%time
ENERGY_MODEL = 'variational_inducing_points' #'variational_inducing_points', 'exact'
ENERGY_NUM_INDUCING_POINTS = 500
ENERGY_LR = 0.01
ENERGY_NUM_STEPS = 2000

FORCES_MODEL = 'variational_inducing_points' #'variational_inducing_points', 'exact'
FORCES_NUM_INDUCING_POINTS = 500
NUM_FORCE_SAMPLES = 1000
FORCES_LR = 0.01
FORCES_NUM_STEPS = 2000

PREDICTOR_NAME = 'inducing.pth'
SUBSAMPLE = 2 # subsample data to reduce time durings tests

! python cook_model.py \
--data_dir $DATA_DIR \
--results_dir $RESULTS_DIR \
--energy_model $ENERGY_MODEL \
--energy_num_inducing_points $ENERGY_NUM_INDUCING_POINTS \
--energy_lr $ENERGY_LR \
--energy_num_steps $ENERGY_NUM_STEPS \
--forces_model $FORCES_MODEL \
--forces_num_inducing_points $FORCES_NUM_INDUCING_POINTS \
--num_force_samples $NUM_FORCE_SAMPLES \
--forces_lr $FORCES_LR \
--forces_num_steps $FORCES_NUM_STEPS \
--predictor_name $PREDICTOR_NAME \
--subsample $SUBSAMPLE

DATA_DIR /content/drive/MyDrive/data/
RESULTS_DIR /content/drive/MyDrive/data/results/
ENERGY_MODEL variational_inducing_points
ENERGY_NUM_INDUCING_POINTS 500
ENERGY_LR 0.01
ENERGY_NUM_STEPS 2000
FORCES_MODEL variational_inducing_points
FORCES_NUM_INDUCING_POINTS 500
NUM_FORCE_SAMPLES 1000
FORCES_LR 0.01
FORCES_NUM_STEPS 2000
PREDICTOR_NAME inducing.pth
500 500
fande.models module imported...
Icecream logger is not available
Total number of found groups: 14
Checking if all atoms are covered:  True
Preparing dataloaders...
Total length of traj is 500
Total number of batches 500
Calculating invariants on trajectory with librascal...
100% 500/500 [00:23<00:00, 21.63it/s]
invariants for energy fitting calculated
100% 500/500 [13:33<00:00,  1.63s/it]
invariants for forces fitting calculated
torch.Size([500, 1200])
torch.Size([500])
Dataloader for group 0 created
Number of samples in dataloader: 1000
Dataloader for group 1 created
Number of samples in dataloader: 1000
Dataloader for group 2 

## Testing models

In [24]:
from ase import io
test_traj = io.read(DATA_DIR + "/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="1000:1010")

In [28]:
import sys
import torch
FANDE_DIR = os.path.expanduser("~/repos/")
sys.path.append(FANDE_DIR + "fande/")

from fande.predict import FandePredictor
from fande.ase import FandeCalc
# load the predictor:
# predictor_loaded = torch.load(RESULTS_DIR + "/fande_predictor.pth")
predictor_loaded = torch.load(RESULTS_DIR + "/inducing.pth")
fande_calc_loaded = FandeCalc(predictor_loaded)
device = torch.device('cpu')
fande_calc_loaded.predictor.move_models_to_device(device)

In [29]:
forces_true = []
forces_predicted = []

energy_true = []
energy_predicted = []

for atoms in test_traj:
    forces_true.append(atoms.get_forces())
    energy_true.append(atoms.get_potential_energy())
    atoms.set_calculator(fande_calc_loaded)
    forces_predicted.append(atoms.get_forces())
    energy_predicted.append(atoms.get_potential_energy())

INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  2516.345262527466
Predicting forces...


INFO:fande:Setting context for descriptors calculation to production


Energy model summary: 
Time invariants:  0.22792816162109375
Time prediction:  16.245365142822266
Time moving on device:  0.3342628479003906
Time total:  18.581628799438477
Time for invariants (call from forces):  1704.0982246398926
Predicting forces...


INFO:fande:Setting context for descriptors calculation to production


...

Time for invariants (call from forces):  1709.369421005249
Predicting forces...
Energy model summary: 
Time invariants:  0.179290771484375
Time prediction:  4.288911819458008
Time moving on device:  0.29087066650390625
Time total:  12.606382369995117


INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1621.034860610962
Predicting forces...
Energy model summary: 
Time invariants:  0.2205371856689453
Time prediction:  21.03877067565918
Time moving on device:  9.911060333251953
Time total:  39.090871810913086


INFO:fande:Setting context for descriptors calculation to production


Time for invariants (call from forces):  1721.6062545776367
Predicting forces...
Energy model summary: 
Time invariants:  0.19478797912597656
Time prediction:  14.042139053344727
Time moving on device:  0.3590583801269531
Time total:  22.502660751342773
Time for invariants (call from forces):  1603.8358211517334
Predicting forces...
Energy model summary: 
Time invariants:  0.2562999725341797
Time prediction:  19.92964744567871
Time moving on device:  1.1000633239746094
Time total:  22.517919540405273


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

energy_true = np.array(energy_true)
energy_predicted = np.array(energy_predicted)
energy_errors = energy_predicted - energy_true

forces_true = np.array(forces_true)
forces_predicted = np.array(forces_predicted)
forces_errors = forces_predicted - forces_true

atomic_groups = predictor_loaded.fdm.atomic_groups



for ag in atomic_groups:

    print("Atomic group", ag)
    print("F_x")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 0].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 0].flatten()**2)))
    print("F_y")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 1].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 1].flatten()**2)))
    print("F_z")
    print("MAE", np.mean(np.abs(forces_errors[:, ag, 2].flatten())))
    print("RMSE", np.sqrt(np.mean(forces_errors[:, ag, 2].flatten()**2)))
    print("E")
    print("MAE", np.mean(np.abs(energy_errors.flatten())))
    print("RMSE", np.sqrt(np.mean(energy_errors.flatten()**2)))

    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))
    axs[0].set_title(r"$F_x$")
    axs[0].plot(np.array(forces_true)[:, ag, 0].flatten(), label="true")
    axs[0].plot(np.array(forces_predicted)[:, ag, 0].flatten(), label="predicted")
    axs[0].legend()

    axs[1].set_title(r"$F_y$")
    axs[1].plot(np.array(forces_true)[:, ag, 1].flatten(), label="true")
    axs[1].plot(np.array(forces_predicted)[:, ag, 1].flatten(), label="predicted")
    axs[1].legend()

    axs[2].set_title(r"$F_z$")
    axs[2].plot(np.array(forces_true)[:, ag, 2].flatten(), label="true")
    axs[2].plot(np.array(forces_predicted)[:, ag, 2].flatten(), label="predicted")
    axs[2].legend()

    plt.show()

    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))

    axs[0].set_title(r"$F_x$ errors")
    axs[0].hist(forces_errors[:, ag, 0].flatten(), bins=100)

    axs[1].set_title(r"$F_y$ errors")
    axs[1].hist(forces_errors[:, ag, 1].flatten(), bins=100)

    axs[2].set_title(r"$F_z$ errors")
    axs[2].hist(forces_errors[:, ag, 2].flatten(), bins=100)
    plt.show()

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 3))
    axs[0].set_title(r"$E$ errors")
    axs[0].hist(energy_errors.flatten(), bins=100)

    axs[1].set_title(r"$E$")
    axs[1].plot(energy_true, label="true")
    axs[1].plot(energy_predicted, label="predicted")
    axs[1].legend()
    plt.show()
